In [1]:
import pandas as pd 
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np 
import argparse
from copy import deepcopy #Add Deepcopy for args

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

1.4.0
Populating the interactive namespace from numpy and matplotlib


# 1. Data loading & Preprocessing

### 1.1 Data loading

In [2]:
cd /Users/jinsungpark/Desktop

/Users/jinsungpark/Desktop


In [3]:
ls #현재경로에 있는 항목 확인

2020_공모전/
3.덕곡-함안_상관계수정리.xlsx
DGHA_ALL_TN/
DGHA_ALL_TP/
DSNG_ALL_TN/
DSNG_ALL_TP/
IMG_0221.png
Input(DG)_data.xlsx
Input(DG)_log.xlsx
Output(HA)_data.xlsx
Output(HA)_log.xlsx
Python/
Real_Last/
RiverLast/
TN(DG_HA)_exp01/
TN(DG_HA)_exp02/
TP(DG_HA)_exp01/
TP(DG_HA)_exp02/
Unity/
corr_p(덕곡_함안).xlsx
corr_s(덕곡_함안).xlsx
etc/
jupyter/
model_save/
~$논문 제출용.docx
과외/
임시/
덕곡_함안.xlsx
발표용.pptx
스크린샷 2020-05-08 오후 10.57.51.png
결과이미지/
시각디자인/
졸업논문/
환경공학/
졸업논문 제출용.docx
표가져오기용.pptx
졸논잉여파일/


In [4]:
UpStream_data = pd.read_excel('Input(DG)_log.xlsx')
DownStream_data = pd.read_excel('Output(HA)_log.xlsx')

In [5]:
# print(UpStream_data.columns)
# print(DownStream_data.columns)

In [6]:
#날짜 인덱스화
UpData = UpStream_data.set_index('Date')
DownData = DownStream_data.set_index('Date')

In [7]:
UpData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 74 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DukGok_DO          349 non-null    float64
 1   DukGok_BOD         349 non-null    float64
 2   DukGok_COD         349 non-null    float64
 3   DukGok_SS          349 non-null    float64
 4   DukGok_TN          349 non-null    float64
 5   DukGok_TP          349 non-null    float64
 6   DukGok_Chl_a       349 non-null    float64
 7   ChilSeo_COD        349 non-null    float64
 8   ChilSeo_SS         349 non-null    float64
 9   ChilSeo_TN         349 non-null    float64
 10  ChilSeo_TP         349 non-null    float64
 11  ChilSeo_Flow_mean  349 non-null    float64
 12  ChilSeo_Flow_day   349 non-null    float64
 13  ChilSeo_COD_load   349 non-null    float64
 14  ChilSeo_SS_load    349 non-null    float64
 15  ChilSeo_TN_load    349 non-null    float64
 16  ChilSeo_TP_load

In [8]:
# #넣고싶은 상류 항목 컬럼 선택 - TN setting(0.25기준)
# UpData = UpData.iloc[:,[0,1,4,67,9,18,19,23,68,27,28,29,31,32,69,33,35,37,38,39,40,70,41,43,47,71,51,53,55,56,72,59,60,63,64,73,65]]
# UpData.info()

In [9]:
#넣고싶은 상류 항목 컬럼 선택 - TN=TP ALL setting
UpData = UpData.drop(['ChilSeo_Flow_day', 'ChilSeo_COD_load', 'ChilSeo_SS_load', 'ChilSeo_TN_load', 'ChilSeo_TP_load'], axis = 1)
UpData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 69 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DukGok_DO          349 non-null    float64
 1   DukGok_BOD         349 non-null    float64
 2   DukGok_COD         349 non-null    float64
 3   DukGok_SS          349 non-null    float64
 4   DukGok_TN          349 non-null    float64
 5   DukGok_TP          349 non-null    float64
 6   DukGok_Chl_a       349 non-null    float64
 7   ChilSeo_COD        349 non-null    float64
 8   ChilSeo_SS         349 non-null    float64
 9   ChilSeo_TN         349 non-null    float64
 10  ChilSeo_TP         349 non-null    float64
 11  ChilSeo_Flow_mean  349 non-null    float64
 12  BukChang_Rain      349 non-null    float64
 13  BukChang_Solar     349 non-null    float64
 14  HwangGang_DO       349 non-null    float64
 15  HwangGang_BOD      349 non-null    float64
 16  HwangGang_COD  

In [10]:
DownData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   HamAn_DO     349 non-null    float64
 1   HamAn_BOD    349 non-null    float64
 2   HamAn_COD    349 non-null    float64
 3   HamAn_SS     349 non-null    float64
 4   HamAn_TN     349 non-null    float64
 5   HamAn_TP     349 non-null    float64
 6   HamAn_Chl_a  349 non-null    float64
dtypes: float64(7)
memory usage: 21.8+ KB


In [11]:
#알고싶은 하류 항목 컬럼 넘버 넣기('Date'항목이 인덱스화 돼서 컬럼 넘버가 -1씩 됨)
Colum = 4
print(DownData.columns[Colum])

HamAn_TN


### 1.2 Data Preprocessing(normalization)

In [12]:
from sklearn.preprocessing import MinMaxScaler

UpScaler = MinMaxScaler() #상류데이터용
DownScaler = MinMaxScaler() #하류데이터용

#나중에 결과를 DeNormalizing 하기 위해 나누어 사용 하였다.

def DeNormalize(Y, Data_name, column_num, Scaler_Type):
    
    data = Data_name
    Scaler = Scaler_Type
    
    _max = Scaler.data_max_[column_num] # 역정규화 하려는 데이터의 컬럼 번호
    _min = Scaler.data_min_[column_num] 
    
    X = Y*(_max-_min) + _min
    
    return X

In [13]:
#데이터 정규화
UpData = pd.DataFrame(UpScaler.fit_transform(UpData), columns=UpData.columns, index=UpData.index)
DownData = pd.DataFrame(DownScaler.fit_transform(DownData), columns=DownData.columns, index=DownData.index)

print(UpData.isna().sum())

DukGok_DO         0
DukGok_BOD        0
DukGok_COD        0
DukGok_SS         0
DukGok_TN         0
                 ..
SanBan_Temp       0
NamGang_Temp      0
GyeaSung_Temp     0
GwangRyeo_Temp    0
ToPyeong_Temp     0
Length: 69, dtype: int64


# 2. Data Preparation

In [14]:
class RiverDataset(Dataset):
    def __init__(self, UpData, DownData, x_frames, y_frames, start, end):
        
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        self.start = start
        self.end = end

        self.UpData = UpData[start:end]
        self.DownData = DownData[start:end]

    def __len__(self):
        return len(self.UpData) - (self.x_frames + self.y_frames) + 1
    #데이터를 전처리 할때 UpData와 DownData의 길이가 동일해짐(날짜를 동일한것만 추출해야 하므로), 따라서 전체길이는 둘중 하나를 사용

    def __getitem__(self, idx):
        idx += self.x_frames

        X = self.UpData.iloc[idx-self.x_frames:idx].values
        Y = self.DownData.iloc[idx:idx+self.y_frames].values
        
        return X, Y

# 3. Model Define

In [15]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers) #
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred

In [16]:
# 정확도 : 예측확률을 100%로 봤을때 MAPE에 따른 오차비율을 빼줌 (100-MAPE) ##RMSE, MAPE 두개로 볼 수 있게
def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [17]:
def RMSE(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    return np.sqrt(mse)

In [18]:
def R2(y_true, y_pred):
    R2_score = r2_score(y_true, y_pred, multioutput='raw_values')
    return R2_score

# 4. Train, Validate, Test

In [19]:
def train(model, partition, optimizer, loss_fn, args):
    trainloader = DataLoader(partition['train'],
                             batch_size=args.batch_size,
                             shuffle=False, drop_last=True)
    model.train()
    model.zero_grad()
    optimizer.zero_grad()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    train_acc = 0.0
    train_loss = 0.0
    
    for i, (X, y) in enumerate(trainloader):

        X = X.transpose(0, 1).float().to(args.device)#파이토치는 순서가 달라서 바꿔줌
        y_true = y[:, :, Colum].float().to(args.device)
        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1), y_true.view(-1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        pred.append(y_pred)
        true.append(y_true)

    # ========================================================================== #
    for i in range(len(trainloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = DeNormalize(tems1[j], DownData, Colum, DownScaler)
            value2 = DeNormalize(tems2[j], DownData, Colum, DownScaler)          
#             value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
#             value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================================================== #   

    train_loss = train_loss / len(trainloader)
    train_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    train_acc2 = R2(np.array(true_results), np.array(pred_results))
#     train_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    return model, train_loss, train_acc1[0], train_acc2[0]

In [20]:
def validate(model, partition, loss_fn, args):
    valloader = DataLoader(partition['val'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    val_acc = 0.0
    val_loss = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):

            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)
            loss = loss_fn(y_pred.view(-1), y_true.view(-1))

            val_loss += loss.item()
            
            pred.append(y_pred)
            true.append(y_true)

        # ========================================================================== #
        for i in range(len(valloader)):
            tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
            tems2 = true[i].view(bat_siz).cpu().detach().numpy()

            for j in range(bat_siz):
                
                value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
                value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))

                pred_results.append(value1)
                true_results.append(value2)
        # ========================================================================== #   

    val_loss = val_loss / len(valloader)
    val_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    val_acc2 = R2(np.array(true_results), np.array(pred_results))
#     val_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    
    return val_loss, val_acc1[0], val_acc2[0]

In [21]:
def test(model, partition, args):
    testloader = DataLoader(partition['test'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    test_acc = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)

            pred.append(y_pred)
            true.append(y_true)

    # =================== test 데이터 시각화를 위해 x,y데이터 저장 =================== #
    for i in range(len(testloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ======================================================================== #   

    test_acc1 =  RMSE(np.array( true_results), np.array(pred_results))
    test_acc2 =  R2(np.array( true_results), np.array(pred_results))
#     test_acc3 =  (100 - MAPE(np.array( true_results), np.array(pred_results)))
    
    return test_acc1[0], test_acc2[0], pred_results, true_results

# 5. Experiment

In [22]:
def experiment(partition, args):

    model = LSTM(args.input_dim, args.hid_dim, args.y_frames, args.n_layers, args.batch_size, args.dropout, args.use_bn)
    model.to(args.device)

    if args.loss == 'MSELoss':
        loss_fn = torch.nn.MSELoss()
        loss_fn = nn.MSELoss()
    elif args.loss == 'L1Loss':
        loss_fn = torch.nn.L1Loss()
        loss_fn = nn.L1Loss()
    elif args.loss == 'PoissonNLLLoss':
        loss_fn = torch.nn.PoissonNLLLoss()
        loss_fn = nn.PoissonNLLLoss()
    elif args.loss == 'KLDivLoss':
        loss_fn = torch.nn.KLDivLoss()
        loss_fn = nn.KLDivLoss()
    elif args.loss == 'BCELoss':
        loss_fn = torch.nn.BCELoss()
        loss_fn = nn.BCELoss()
    elif args.loss == 'BCEWithLogitsLoss':
        loss_fn = torch.nn.BCEWithLogitsLoss()
        loss_fn = nn.BCEWithLogitsLoss()
    else:
        raise ValueError('In-valid LossFuction choice')
    
    if args.optim == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    train_accs_RMSE = []
    train_accs_R2 = []
    val_accs_RMSE = []
    val_accs_R2 = []
    result_info = pd.DataFrame()
    result_data = pd.DataFrame()
    
    # ===================================== #
    
    ## model starting point ##    
    ts = time.time()
    model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
    val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
    te = time.time()

    # ====== Add Epoch Data ====== #
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs_RMSE.append(train_acc_RMSE)
    val_accs_RMSE.append(val_acc_RMSE)
    train_accs_R2.append(train_acc_R2)
    val_accs_R2.append(val_acc_R2)
    # ============================ #


#     print('Epoch {}, Acc_RMSE(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
#           .format(0, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
    
    for epoch in range(args.epoch-1):  # loop over the dataset multiple times
        
        ts = time.time()
        model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
        val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
        te = time.time()

        # ====== Add Epoch Data ====== #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs_RMSE.append(train_acc_RMSE)
        val_accs_RMSE.append(val_acc_RMSE)
        train_accs_R2.append(train_acc_R2)
        val_accs_R2.append(val_acc_R2)
        # ============================ #
        
        
        if epoch == epoch:
            trash_01, trash_02, Pred_save, True_save = test(model, partition, args)
            result_data['test_pred_epoch({})'.format(epoch)] = Pred_save
            result_data['test_true_epoch({})'.format(epoch)] = True_save


#         print('Epoch {}, Acc_RMSE(train/val): {:2.4f}/{:2.4f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
#               .format(epoch+1, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
        
    test_acc_RMSE, test_acc_R2, Pred_data, True_data = test(model, partition, args)    
    
    # ======= Add Result  ======= #
    result_info['train_losses'] = train_losses
    result_info['val_losses'] = val_losses
    
    result_info['train_accs_RMSE'] = train_accs_RMSE
    result_info['train_accs_R2'] = train_accs_R2
    result_info['val_accs_RMSE'] = val_accs_RMSE
    result_info['val_accs_R2'] = val_accs_R2
    
#     result_info['train_acc'] = train_acc
#     result_info['val_acc'] = val_acc
    result_info['test_RMSE'] = test_acc_RMSE
    result_info['test_R2'] = test_acc_R2

#     result_data['test_pred'] = Pred_data
#     result_data['test_true'] = True_data
    
    return result_info, result_data

# 6. LSTM Run

In [23]:
# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ====== Data Loading ====== #
args.batch_size = 8
args.UpData = UpData
args.DownData = DownData
args.x_frames = 5
args.y_frames = 1

# ====== Model Capacity ===== #
args.input_dim = len(UpData.columns)
args.hid_dim = 16
args.n_layers = 2

# ====== Regularization ======= #
args.l2 = 0.0001
args.dropout = 0.1 
args.use_bn = False

# ====== Optimizer & Training ====== #
args.optim = 'Adam'  #SGD, RMSprop, Adam...
args.loss = 'MSELoss'#'MSELoss','L1Loss','PoissonNLLLoss','KLDivLoss','BCELoss','BCEWithLogitsLoss'
args.lr = 0.01
args.epoch = 3


# ====== Experiment Variable ====== #
name_var1 = 'x_frames'
list_var1 = [4]

name_var2 = 'loss'
list_var2 = ['MSELoss']

name_var3 = 'optim'
list_var3 = ['Adam']

name_var4 = 'use_bn'
list_var4 = [False]

name_var5 = 'dropout'
list_var5 = [0.1]

name_var6 = 'batch_size'
list_var6 = [4, 8]

name_var7 = 'hid_dim'
list_var7 = [32, 16]

name_var8 = 'n_layers'
list_var8 = [4, 3, 2]

name_var9 = 'lr'
list_var9 = [0.01, 0.001, 0.0001]

name_var10 = 'l2'
list_var10 = [0.01, 0.001,0.0001]

name_var11 = 'epoch'
list_var11 = [200]

trainset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2013-01-01', '2018-06-30')
valset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2018-07-01', '2019-12-31')
testset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2018-07-01', '2019-12-31')
partition = {'train': trainset, 'val':valset, 'test':testset}

print(' size of trainset :{}\n'.format(len(trainset)),
      'size of valset :{}\n'.format(len(valset)),
      'size of testset :{}'.format(len(testset)))

list_vars = [list_var1, list_var2, list_var3, list_var4, list_var5, list_var6, list_var7, list_var8, list_var9, list_var10, list_var11]
i = 1
for lenth in list_vars:
    x = len(lenth)
    i *= x
total_exp_num = i

print(' total_exp_num : {}'.format(total_exp_num))

 size of trainset :279
 size of valset :60
 size of testset :60
 total_exp_num : 108


In [24]:
pwd

'/Users/jinsungpark/Desktop'

In [ ]:
cd /Users/jinsungpark/Desktop/DGHA_ALL_TN

In [26]:
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

print('##### Start #####')

num = 0 #초기화

for var1 in list_var1:
    for var2 in list_var2:
        for var3 in list_var3:
            for var4 in list_var4:
                for var5 in list_var5:
                    for var6 in list_var6:
                        for var7 in list_var7:
                            for var8 in list_var8:
                                for var9 in list_var9:
                                    for var10 in list_var10:
                                        for var11 in list_var11:
                                            ts = time.time()
                                            num += 1
                                            setattr(args, name_var1, var1)
                                            setattr(args, name_var2, var2)
                                            setattr(args, name_var3, var3)
                                            setattr(args, name_var4, var4)
                                            setattr(args, name_var5, var5)
                                            setattr(args, name_var6, var6)
                                            setattr(args, name_var7, var7)
                                            setattr(args, name_var8, var8)
                                            setattr(args, name_var9, var9)
                                            setattr(args, name_var10, var10)
                                            setattr(args, name_var11, var11)

#                                             print('experiment_{}/{} : x_frames = {}, loss={}, optim={}, use_bn={}, dropout={}, batch_size={}, hid_dim={}, n_layers={}, lr={}, l2={}, epoch={}'
#                                                   .format(num,total_exp_num,args.x_frames,args.loss,args.optim,args.use_bn,args.dropout,args.batch_size,args.hid_dim,args.n_layers,args.lr,args.l2,args.epoch))
                                            result_info, result_data = experiment(partition, deepcopy(args))
    
                                            min_RMSE = min(result_info['val_accs_RMSE'])
                                            max_R2 = max(result_info['val_accs_R2'])
#                                             RMSE_ = result_info['val_accs_RMSE']
#                                             R2_ = result_info['val_accs_R2']
                                            epoch = args.epoch
                                            
                                            result_info.to_csv('exp{:03d}_info_RMSE[{:2.4f}]_R2[{:2.4f}].csv'.format(num,min_RMSE,max_R2))
                                            result_data.to_csv('exp{:03d}_data_RMSE[{:2.4f}]_R2[{:2.4f}].csv'.format(num,min_RMSE,max_R2))
                
                                            file=open('exp_index.txt','a')
                                            file.write('experiment {:03d}/{} : x_frames = {}, loss={}, optim={}, use_bn={}, dropout={}, batch_size={}, hid_dim={}, n_layers={}, lr={}, l2={}, epoch={}\n'
                                                       .format(num,total_exp_num,args.x_frames,args.loss,args.optim,args.use_bn,args.dropout,args.batch_size,args.hid_dim,args.n_layers,args.lr,args.l2,args.epoch))
                                            file.close()

                                            te = time.time()
                                
                                            print('experiment_{}/{}, took {:2.1f}sec, {:2.2f}% done'
                                                  .format(num,total_exp_num,te-ts,(num/total_exp_num)*100))
                                    
#                                             network = LSTM(args.input_dim, args.hid_dim, args.y_frames, args.n_layers, args.batch_size, args.dropout, args.use_bn)
#                                             torch.save(network.state_dict(),'lstm1.pt')
                            
                                       
#                                             print('train_acc = {:2.2f}%, val_acc = {:2.2f}%, test_acc = {:2.2f}%'
#                                                   .format(result['train_acc'],result['val_acc'],result['test_acc']))

print('All done')

##### Start #####
experiment_1/108, took 306.0sec, 0.93% done
experiment_2/108, took 250.0sec, 1.85% done
experiment_3/108, took 238.2sec, 2.78% done
experiment_4/108, took 223.4sec, 3.70% done
experiment_5/108, took 162.0sec, 4.63% done
experiment_6/108, took 164.6sec, 5.56% done
experiment_7/108, took 251.5sec, 6.48% done
experiment_8/108, took 243.0sec, 7.41% done
experiment_9/108, took 240.2sec, 8.33% done
experiment_10/108, took 249.0sec, 9.26% done
experiment_11/108, took 245.9sec, 10.19% done
experiment_12/108, took 240.5sec, 11.11% done
experiment_13/108, took 274.5sec, 12.04% done
experiment_14/108, took 201.8sec, 12.96% done
experiment_15/108, took 201.7sec, 13.89% done
experiment_16/108, took 201.0sec, 14.81% done
experiment_17/108, took 199.7sec, 15.74% done
experiment_18/108, took 206.4sec, 16.67% done
experiment_19/108, took 189.1sec, 17.59% done
experiment_20/108, took 193.1sec, 18.52% done
experiment_21/108, took 163.0sec, 19.44% done
experiment_22/108, took 162.1sec, 2

In [ ]:
# 원래데이터 백업

# # ====== Random Seed Initialization ====== #
# seed = 666
# np.random.seed(seed)
# torch.manual_seed(seed)

# parser = argparse.ArgumentParser()
# args = parser.parse_args("")

# args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# # ====== Data Loading ====== #
# args.batch_size = 8
# args.UpData = UpData
# args.DownData = DownData
# args.x_frames = 5
# args.y_frames = 1

# # ====== Model Capacity ===== #
# args.input_dim = len(UpData.columns)
# args.hid_dim = 16
# args.n_layers = 2

# # ====== Regularization ======= #
# args.l2 = 0.0001
# args.dropout = 0.1 
# args.use_bn = False

# # ====== Optimizer & Training ====== #
# args.optim = 'Adam'  #SGD, RMSprop, Adam...
# args.loss = 'MSELoss'#'MSELoss','L1Loss','PoissonNLLLoss','KLDivLoss','BCELoss','BCEWithLogitsLoss'
# args.lr = 0.01
# args.epoch = 3


# # ====== Experiment Variable ====== #
# name_var1 = 'x_frames'
# list_var1 = [4,2]

# name_var2 = 'loss'
# list_var2 = ['MSELoss']

# name_var3 = 'optim'
# list_var3 = ['Adam']

# name_var4 = 'use_bn'
# list_var4 = [False]

# name_var5 = 'dropout'
# list_var5 = [0.1]

# name_var6 = 'batch_size'
# list_var6 = [4, 8]

# name_var7 = 'hid_dim'
# list_var7 = [32, 16]

# name_var8 = 'n_layers'
# list_var8 = [4, 3, 2]

# name_var9 = 'lr'
# list_var9 = [0.01, 0.001, 0.0001]

# name_var10 = 'l2'
# list_var10 = [0.01, 0.001,0.0001]

# name_var11 = 'epoch'
# list_var11 = [200]

# trainset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2013-01-01', '2018-06-30')
# valset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2018-07-01', '2019-12-31')
# testset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2018-07-01', '2019-12-31')
# partition = {'train': trainset, 'val':valset, 'test':testset}

# print(' size of trainset :{}\n'.format(len(trainset)),
#       'size of valset :{}\n'.format(len(valset)),
#       'size of testset :{}'.format(len(testset)))

# list_vars = [list_var1, list_var2, list_var3, list_var4, list_var5, list_var6, list_var7, list_var8, list_var9, list_var10, list_var11]
# i = 1
# for lenth in list_vars:
#     x = len(lenth)
#     i *= x
# total_exp_num = i

# print(' total_exp_num : {}'.format(total_exp_num))